In [ ]:
import pickle, os, sys, math

import StaticWindow as sw
import ZeroCrossing as zcr
import AE as ae

from scipy.optimize import curve_fit
from scipy.integrate import quad

import numpy as np

import matplotlib.pyplot as plt

## Load Data

In [ ]:
directory = 'INSERT DIRECTORY' # Directory where events generated by Simulation.py are saved.

background_file = '0_SNR_data.pkl' # Baseline eventless data filename.

files = [] # List of simulation filenames

SNRs = [float(x.split('_')[0]) for x in files]

# Load Data and Ground Truths
raw_traces, ground_truths = sw.load_SNR_data(directory, files)

thresholds = [0, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 0.9999, 1]



In [ ]:
# Load Background data
with open(os.path.join(directory, background_file), 'rb') as f:
    background_data = pickle.load(f)[0]

## Comparison of standard error

In [ ]:
def generate_hist_counts(all_data, sub_data, bins=10):
    bin_edges = np.histogram_bin_edges(all_data, bins=bins)
    counts, _ = np.histogram(sub_data, bins=bin_edges)
    return bin_edges, counts

def gauss(x, A, mu, sig):
    B = ((-(x - mu) ** 2) / 2 *(sig ** 2))
    return A * np.exp(B)
    
def generate_gauss_params(bin_mids, counts, data):
    params, cov = curve_fit(gauss, bin_mids, counts, p0=(counts.max(), data.mean(), data.std()))
    norm_factor = quad(gauss, -np.inf, np.inf, args=tuple(params))[0]
    
    return params, norm_factor

### Background

In [ ]:
bins = 1000
# Scale background data
background_data = background_data * 1000

In [ ]:
# Calculate background distribution
bin_edges, back_counts = generate_hist_counts(background_data.flatten(), background_data.flatten(), bins=bins)
bin_mids = bin_edges[1:] - (np.diff(bin_edges) / 2)
gauss_params, norm_factor = generate_gauss_params(bin_mids, back_counts, background_data.flatten())

gauss_params_norm = gauss_params.copy()
gauss_params_norm[0] = gauss_params_norm[0] / norm_factor

fig, ax = plt.subplots()
h1 = ax.bar(bin_mids, back_counts / norm_factor, width=np.diff(bin_edges))
p1 = ax.plot(bin_mids, gauss(bin_mids, *gauss_params_norm), color='orange')

In [ ]:
# Calcualte background distribution with moving average
# Then compare with previous distribution
background_means = sw.calculate_means(background_data, 100, 100)

back_mean_counts, _ = np.histogram(background_means, bins=bin_edges)
mean_gauss_params, mean_norm_factor = generate_gauss_params(bin_mids, back_mean_counts, background_means.flatten())

mean_gauss_params_norm = mean_gauss_params.copy()
mean_gauss_params_norm[0] = mean_gauss_params_norm[0] / mean_norm_factor

fig, ax = plt.subplots(figsize=(6, 4), dpi=600)
ax.plot(bin_mids, gauss(bin_mids, *gauss_params_norm))
ax.plot(bin_mids, gauss(bin_mids, *mean_gauss_params_norm))
ax.set_xlabel('Current / pA', weight='bold')
ax.set_ylabel('Normalised Counts', weight='bold')
ax.set(xlim=(-6, 6))
ax.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax.spines[axis].set_linewidth(1.5)
    
fig.savefig("Standard Error.png")

### Background vs Events
Demonstrate the separability between background and events with larger sampling size

In [ ]:
snr = 0.1
bins = 2000

window_size = 2000
step_size = 2000

traces = raw_traces[snr] * 1000
evt_locs = ground_truths[snr]
background_points = traces[evt_locs == 0]
evt_points = traces[evt_locs == 1]

In [ ]:
# Overlap without moving window
bin_edges, back_counts = generate_hist_counts(traces, background_points, bins=bins)
bin_edges, evt_counts = generate_hist_counts(traces, evt_points, bins=bins)
bin_mids = bin_edges[1:] - (np.diff(bin_edges) / 2)

back_gauss_params, back_norm_factor = generate_gauss_params(bin_mids, back_counts, background_points)
evt_gauss_params, evt_norm_factor = generate_gauss_params(bin_mids, evt_counts, evt_points)

fig, ax = plt.subplots()
h1 = ax.bar(bin_mids, back_counts / back_norm_factor, width=np.diff(bin_edges))
h2 = ax.bar(bin_mids, evt_counts / evt_norm_factor, width=np.diff(bin_edges))
ax.set(title='Point Distribtuions')
ax.legend(['Background', 'Events'])

In [ ]:
# Overlap with moving window
background_means = sw.calculate_means([background_points], window_size, step_size)
evt_means = sw.calculate_means([evt_points], window_size, step_size)

bin_edges, back_counts = generate_hist_counts(traces, background_means, bins=bins)
bin_edges, evt_counts = generate_hist_counts(traces, evt_means, bins=bins)
bin_mids = bin_edges[1:] - (np.diff(bin_edges) / 2)

back_gauss_params, back_norm_factor = generate_gauss_params(bin_mids, back_counts, background_points)
evt_gauss_params, evt_norm_factor = generate_gauss_params(bin_mids, evt_counts, evt_points)

fig, ax = plt.subplots()
h1 = ax.bar(bin_mids, back_counts / back_norm_factor, width=np.diff(bin_edges))
h2 = ax.bar(bin_mids, evt_counts / evt_norm_factor, width=np.diff(bin_edges))
ax.set(title='Point Distribtuions')
ax.legend(['Background', 'Events'])